In [ ]:
import os
import sys
import subprocess

# --- CONFIGURAZIONE DEL PROGETTO ---
# Modifica queste variabili per clonare il tuo repository specifico
GIT_REPO_URL = "https://github.com/Biobay/DeepLearning/"  # <-- INSERISCI QUI L'URL DEL TUO REPOSITORY
BRANCH_NAME = "RIS-1-CORRETTO"  # <-- INSERISCI QUI IL NOME DEL BRANCH
PROJECT_DIR = "." # <-- NOME DELLA CARTELLA DOPO LA CLONAZIONE

# --- SCRIPT DI SETUP E TRAINING ---

# 1. Clona il repository e si sposta nel branch specificato
# NOTA: Questo script ora presuppone che venga eseguito dalla cartella principale del progetto.
# Se la cartella è vuota, clonerà il repo qui.
if not os.path.exists("scripts"): # Controlla se una cartella chiave del progetto esiste
    print(f"Clonazione del branch '{BRANCH_NAME}' da '{GIT_REPO_URL}'...")
    # Clona in una cartella temporanea e sposta i file
    TEMP_DIR = "temp_clone"
    subprocess.run(['git', 'clone', '--branch', BRANCH_NAME, GIT_REPO_URL, TEMP_DIR], check=True)
    for item in os.listdir(TEMP_DIR):
        os.rename(os.path.join(TEMP_DIR, item), os.path.join(".", item))
    os.rmdir(TEMP_DIR)
else:
    print(f"La cartella del progetto sembra esistere già. Salto la clonazione.")

# 2. Naviga nella cartella del progetto
# os.chdir(PROJECT_DIR) # RIMOSSO - Non cambiamo più la directory
print(f"Cartella di lavoro corrente: {os.getcwd()}")

# 3. Installa le dipendenze
print("Installazione delle dipendenze da requirements.txt...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

# 4. Aggiunge il percorso al path di Python per gli import
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
print("Path di sistema aggiornato.")

# 5. Avvia l'addestramento
print("--- Avvio del processo di addestramento ---")
try:
    from scripts.train import train
    import src.config as config
    print("Moduli del progetto importati con successo!")
    train(config)
    print("--- Processo di addestramento terminato ---")
except ImportError as e:
    print(f"ERRORE: Impossibile importare i moduli. {e}")
except Exception as e:
    print(f"Si è verificato un errore durante l'addestramento: {e}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# --- VISUALIZZAZIONE MIGLIORATA DELLE LOSS (MEDIA PER EPOCA) ---

# 1. Definisci il percorso del file di log
# Il percorso è relativo alla cartella da cui viene eseguito il notebook.
LOG_FILE_PATH = os.path.join("DeepLearning", "results", "logs", "loss_log.csv")

# 2. Controlla se il file di log esiste
if os.path.exists(LOG_FILE_PATH):
    print(f"Trovato file di log in: {LOG_FILE_PATH}")
    
    # 3. Carica i dati e calcola la media per epoca
    df_loss_raw = pd.read_csv(LOG_FILE_PATH)
    df_loss_epoch = df_loss_raw.groupby('epoch').mean()
    print("Calcolata la media delle loss per ogni epoca.")
    
    # 4. Crea il grafico
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax1 = plt.subplots(figsize=(15, 7))

    # Plotta le loss medie (D e G)
    ax1.plot(df_loss_epoch.index, df_loss_epoch['loss_d'], 'o-', label='Avg Discriminator Loss', color='red', alpha=0.8)
    ax1.plot(df_loss_epoch.index, df_loss_epoch['loss_g'], 'o-', label='Avg Generator Loss', color='blue', alpha=0.8)
    
    # Plotta le componenti medie della loss del generatore
    ax1.plot(df_loss_epoch.index, df_loss_epoch['loss_g_adv'], '--', label='Avg Generator Loss (Adversarial)', color='cyan', alpha=0.7)
    ax1.plot(df_loss_epoch.index, df_loss_epoch['loss_g_l1'], '--', label='Avg Generator Loss (L1)', color='lightblue', alpha=0.7)

    # Impostazioni del grafico
    ax1.set_title('Andamento Medio delle Loss per Epoca (Stage-I)', fontsize=16)
    ax1.set_xlabel('Epoch Number', fontsize=12)
    ax1.set_ylabel('Average Loss Value', fontsize=12)
    ax1.legend(loc='upper right')
    ax1.grid(True)
    ax1.set_xticks(df_loss_epoch.index)

    # Migliora la leggibilità degli assi
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # 5. Salva il grafico
    plot_path = os.path.join("DeepLearning", "results", "loss_plot_avg_epoch.png")
    plt.savefig(plot_path)
    print(f"Grafico delle loss medie salvato in: {plot_path}")
    
    # Mostra il grafico
    plt.show()

else:
    print(f"ERRORE: File di log non trovato in '{LOG_FILE_PATH}'. Esegui prima la cella di training.")


### Fase 2: Addestramento StackGAN (Stage-II)

Questa cella avvia lo script di training per la seconda fase, che genera immagini a 256x256 partendo da quelle a 64x64.

In [ ]:
%run scripts/train_s2.py

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

log_file_s2 = 'loss_log_s2.csv'

try:
    df_s2 = pd.read_csv(log_file_s2)
    
    # Calcola la media per epoca se ci sono più voci per la stessa epoca
    df_s2_agg = df_s2.groupby('epoch').mean().reset_index()

    plt.figure(figsize=(12, 6))
    plt.plot(df_s2_agg['epoch'], df_s2_agg['d_loss'], label='Discriminator S2 Loss', color='blue')
    plt.plot(df_s2_agg['epoch'], df_s2_agg['g_loss'], label='Generator S2 Loss', color='green')
    plt.title('Loss del Discriminatore e del Generatore - Fase II')
    plt.xlabel('Epoca')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()
except FileNotFoundError:
    print(f"Il file di log '{log_file_s2}' non è stato trovato. Esegui prima il training della Fase II.")
